# Veri Yükleme ve Ön İşleme

Bu notebook, beyin inmesi CT görüntülerinin yüklenmesi ve ön işlenmesi için gerekli kodları içerir.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import nibabel as nib
from scipy import ndimage
import warnings
warnings.filterwarnings('ignore')

# Görselleştirme ayarları
plt.style.use('default')
sns.set_palette('husl')

print("Kütüphaneler başarıyla yüklendi.")

## Veri Seti Sınıfı (Dataset Class)

In [ ]:
class BrainStrokeDataset(Dataset):
    """
    Beyin inmesi CT görüntüleri için PyTorch Dataset sınıfı
    """
    
    def __init__(self, image_dir, mask_dir, transform=None, augment=False):
        """
        Args:
            image_dir (str): CT görüntülerinin bulunduğu klasör
            mask_dir (str): Segmentasyon maskelerinin bulunduğu klasör
            transform (callable, optional): Görüntülere uygulanacak dönüşümler
            augment (bool): Veri artırma uygulanıp uygulanmayacağı
        """
        self.image_dir = Path(image_dir)
        self.mask_dir = Path(mask_dir)
        self.transform = transform
        self.augment = augment
        
        # Görüntü dosyalarını listele
        self.image_files = sorted(list(self.image_dir.glob('*.png')) + 
                                 list(self.image_dir.glob('*.jpg')) +
                                 list(self.image_dir.glob('*.nii.gz')))
        
        print(f"Toplam {len(self.image_files)} görüntü bulundu.")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        # Görüntü dosyasını yükle
        img_path = self.image_files[idx]
        
        if img_path.suffix == '.gz':  # NIfTI dosyası
            img = nib.load(img_path).get_fdata()
            if img.ndim == 3:  # 3D ise orta slice'ı al
                img = img[:, :, img.shape[2]//2]
        else:  # PNG/JPG dosyası
            img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)
        
        # Mask dosyasını yükle
        mask_path = self.mask_dir / (img_path.stem + '_mask.png')
        if mask_path.exists():
            mask = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
        else:
            mask = np.zeros_like(img)
        
        # Normalizasyon
        img = self.normalize_ct_image(img)
        mask = (mask > 127).astype(np.float32)  # Binary mask
        
        # Tensor'e çevir
        img = torch.from_numpy(img).unsqueeze(0).float()  # (1, H, W)
        mask = torch.from_numpy(mask).unsqueeze(0).float()  # (1, H, W)
        
        # Transform uygula
        if self.transform:
            img = self.transform(img)
        
        return img, mask
    
    def normalize_ct_image(self, img):
        """
        CT görüntüsünü Hounsfield Unit (HU) değerlerine göre normalize et
        """
        # CT değerlerini -1000 (hava) ile 400 (kemik) arasında kırp
        img = np.clip(img, -1000, 400)
        
        # 0-1 aralığına normalize et
        img = (img + 1000) / 1400
        
        return img.astype(np.float32)

## Veri Artırma (Data Augmentation)

In [ ]:
def get_augmentation_transforms():
    """
    Eğitim için veri artırma dönüşümleri
    """
    return transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=10),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
        transforms.Normalize(mean=[0.5], std=[0.5])  # -1 ile 1 arasına normalize et
    ])

def get_validation_transforms():
    """
    Doğrulama için basit normalizasyon
    """
    return transforms.Compose([
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

## Veri Seti Yükleme ve Bölme

In [ ]:
# Veri yolları
dataset_root = Path('./dataset')
image_dir = dataset_root / 'images'
mask_dir = dataset_root / 'masks'

# Test için örnek veri oluştur (gerçek veri yoksa)
def create_sample_data():
    """
    Test için örnek veri oluştur
    """
    image_dir.mkdir(exist_ok=True)
    mask_dir.mkdir(exist_ok=True)
    
    for i in range(5):
        # Örnek CT görüntüsü (512x512)
        img = np.random.randint(0, 255, (512, 512), dtype=np.uint8)
        
        # Örnek lezyon (dairesel)
        mask = np.zeros((512, 512), dtype=np.uint8)
        center = (256 + np.random.randint(-100, 100), 256 + np.random.randint(-100, 100))
        radius = np.random.randint(20, 80)
        cv2.circle(mask, center, radius, 255, -1)
        
        # Dosyalara kaydet
        cv2.imwrite(str(image_dir / f'sample_{i:03d}.png'), img)
        cv2.imwrite(str(mask_dir / f'sample_{i:03d}_mask.png'), mask)
    
    print("Örnek veri oluşturuldu.")

# Gerçek veri yoksa örnek veri oluştur
if not image_dir.exists() or len(list(image_dir.glob('*'))) == 0:
    create_sample_data()

In [ ]:
# Dataset'leri oluştur
train_dataset = BrainStrokeDataset(
    image_dir=image_dir,
    mask_dir=mask_dir,
    transform=get_augmentation_transforms(),
    augment=True
)

val_dataset = BrainStrokeDataset(
    image_dir=image_dir,
    mask_dir=mask_dir,
    transform=get_validation_transforms(),
    augment=False
)

# DataLoader'ları oluştur
batch_size = 4
num_workers = 2

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

print(f"Eğitim dataset boyutu: {len(train_dataset)}")
print(f"Doğrulama dataset boyutu: {len(val_dataset)}")
print(f"Batch boyutu: {batch_size}")

## Veri Görselleştirme

In [ ]:
def visualize_batch(dataloader, num_samples=4):
    """
    Bir batch'ten örnek görüntüleri görselleştir
    """
    dataiter = iter(dataloader)
    images, masks = next(dataiter)
    
    fig, axes = plt.subplots(2, num_samples, figsize=(15, 8))
    
    for i in range(min(num_samples, images.shape[0])):
        # Görüntüyü göster
        img = images[i, 0].cpu().numpy()
        axes[0, i].imshow(img, cmap='gray')
        axes[0, i].set_title(f'CT Görüntüsü {i+1}')
        axes[0, i].axis('off')
        
        # Maski göster
        mask = masks[i, 0].cpu().numpy()
        axes[1, i].imshow(mask, cmap='hot')
        axes[1, i].set_title(f'Segmentasyon Maskesi {i+1}')
        axes[1, i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Örnek batch görselleştir
visualize_batch(train_loader, num_samples=4)

## Veri İstatistikleri

In [ ]:
def analyze_dataset_statistics(dataset):
    """
    Dataset istatistiklerini analiz et
    """
    pixel_values = []
    mask_ratios = []
    
    for i in range(len(dataset)):
        img, mask = dataset[i]
        
        # Piksel değer istatistikleri
        pixel_values.extend(img.flatten().numpy())
        
        # Lezyon oranı
        mask_ratio = mask.sum() / mask.numel()
        mask_ratios.append(mask_ratio.item())
    
    # İstatistikleri göster
    pixel_values = np.array(pixel_values)
    mask_ratios = np.array(mask_ratios)
    
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # Piksel değer dağılımı
    axes[0].hist(pixel_values, bins=50, alpha=0.7, edgecolor='black')
    axes[0].set_title('Piksel Değer Dağılımı')
    axes[0].set_xlabel('Piksel Değeri')
    axes[0].set_ylabel('Frekans')
    
    # Lezyon oranı dağılımı
    axes[1].hist(mask_ratios, bins=20, alpha=0.7, edgecolor='black', color='orange')
    axes[1].set_title('Lezyon Oranı Dağılımı')
    axes[1].set_xlabel('Lezyon Oranı')
    axes[1].set_ylabel('Frekans')
    
    # Box plot
    axes[2].boxplot([mask_ratios], labels=['Lezyon Oranı'])
    axes[2].set_title('Lezyon Oranı İstatistikleri')
    axes[2].set_ylabel('Oran')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Ortalama piksel değeri: {pixel_values.mean():.4f}")
    print(f"Piksel değeri std: {pixel_values.std():.4f}")
    print(f"Ortalama lezyon oranı: {mask_ratios.mean():.4f}")
    print(f"Lezyon oranı std: {mask_ratios.std():.4f}")

# Dataset istatistiklerini analiz et
analyze_dataset_statistics(train_dataset)

## Sonraki Adımlar

Bu notebook'ta veri yükleme ve ön işleme kodlarını geliştirdik. Sonraki notebook'larda:

1. **Topolojik özellik çıkarımı** (GUDHI/gtda kullanarak)
2. **U-Net ve hibrit model mimarileri**
3. **Model eğitimi ve değerlendirmesi**
4. **Sonuçların karşılaştırmalı analizi**

konularını ele alacağız.